# Import Module

In [8]:
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
print(os.listdir('../dataset/preprocessing'))

['Preprocess_test.csv', 'Preprocess_train.csv', 'test.csv', 'train.csv']


# Load Dataset

In [6]:
IMPORT_PATH = '../dataset/preprocessing/'

In [11]:
train = pd.read_csv(IMPORT_PATH + 'Preprocess_train.csv')
test = pd.read_csv(IMPORT_PATH + 'Preprocess_test.csv')

# Check Dataset

In [12]:
print('Train Dataset Shape: ', train.shape)
train.head()

Train Dataset Shape:  (19693, 60)


,gameId,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,blueDeath,...,D_[]['FIRE_DRAGON'],D_[]['WATER_DRAGON'],D_[][],T_['BOT_LANE'][],T_['MID_LANE'][],T_['TOP_LANE'][],T_[]['BOT_LANE'],T_[]['MID_LANE'],T_[]['TOP_LANE'],T_[][]
0,4204710612,19975,3374,35,7.0,216,38,0,13,6,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4153551749,15892,4012,36,7.2,215,57,0,5,4,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4139787761,16270,2485,35,7.0,227,56,0,5,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4202227127,16605,3290,34,6.8,194,28,0,8,10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4189344068,14980,665,33,6.6,199,56,0,4,13,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# TO-DO